Составление отчета по городской логистике. Анализ времени прибытия груза с момента возникновения заявки.

* Таблицы с полученными данными были скрыты. Оставлен скрипт автоматизации расчетов.

Выгрузим необходимые библиотеки для работы с данными

In [1]:
import pandas as pd
import datetime as dt
import datetime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
from numpy import sqrt, abs, round
data = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'Sheet1')

Удалим дубликаты

In [2]:
data = data.drop_duplicates(subset=['№ ЭР'])

Откроем все вкладки в существующем датасете

In [3]:
data_actual = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'актуализация')
data_sr_dost_hr = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'сроки дост хран')
data_dop_scl = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'доп складыДО')
data_div = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'див')
data_hour_p = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'час пояса')
data_period = pd.read_excel('Отчет ГЛ за 22-28.05.XLSX', sheet_name = 'периоды')

Соединим вкладки основной выгрузки и часового пояса

In [4]:
data_1 = pd.merge(data, data_hour_p, on ='Наименование завода отправителя', how ='left')
data_1 = data_1.drop(['Часовой пояс'], axis=1)
data_1 = data_1.drop(['Значение'], axis=1)
data_1 = data_1.drop(['формула дельта в часах ко времени ЕКБ'], axis=1)
data_1['дельта в часах ко времени ЕКБ'] = data_1['дельта в часах ко времени ЕКБ'].abs()

Еще раз преобразуем предыдущее слияние датасетов, удалив ненужные для расчета столбцы

In [5]:
data_2 = pd.merge(data_1, data_hour_p, on ='Наименование завода отправителя', how ='left')
data_2 = data_2.drop(['Часовой пояс'], axis=1)
data_2 = data_2.drop(['формула дельта в часах ко времени ЕКБ'], axis=1)
data_2 = data_2.drop(['дельта в часах ко времени ЕКБ_y'], axis=1)
data_2 = data_2.rename(columns={'Значение' : 'Прибавить'})

Произведем преобразования в столбцах со временем с возможностью добавления значений формата int к исходному datetime

In [6]:
data_2['дельта в часах ко времени ЕКБ_x'] = data_2['дельта в часах ко времени ЕКБ_x'].astype(int)
data_2['time'] = data_2['Дата создания'].astype(str) + " " + data_2['Время'].astype(str)
data_2['time'] = pd.to_datetime(data_2['time'])

In [7]:
data_2['time']

0       2023-05-28 16:07:56
1       2023-05-28 13:05:39
2       2023-05-24 10:45:19
3       2023-05-23 10:16:39
4       2023-05-22 14:52:49
                ...        
12616   2023-05-25 13:04:44
12617   2023-05-25 15:20:48
12618   2023-05-26 10:55:25
12619   2023-05-27 00:08:07
12620   2023-05-25 14:30:27
Name: time, Length: 12621, dtype: datetime64[ns]

Произведем действия по добавлению (вычитанию) времени из столбцов даты и времени прибытия груза

In [8]:
def time_lp(data_2):
    if data_2['Прибавить'] == 'прибавить':
        return data_2['time'] + relativedelta(hours=data_2['дельта в часах ко времени ЕКБ_x'])
    else:
        return data_2['time'] - relativedelta(hours=data_2['дельта в часах ко времени ЕКБ_x'])
data_2['дата и время создания с учетом час пояса'] = data_2.apply(time_lp, axis=1)
data_2['дата и время создания с учетом час пояса']

0       2023-05-28 16:07:56
1       2023-05-28 11:05:39
2       2023-05-24 09:45:19
3       2023-05-23 10:16:39
4       2023-05-22 12:52:49
                ...        
12616   2023-05-25 11:04:44
12617   2023-05-25 15:20:48
12618   2023-05-26 10:55:25
12619   2023-05-26 22:08:07
12620   2023-05-25 14:30:27
Name: дата и время создания с учетом час пояса, Length: 12621, dtype: datetime64[ns]

Произведем обратный перевод из числового формата int в datetime

In [9]:
data_2['Дата создания с час поясом'] = pd.to_datetime(data_2['дата и время создания с учетом час пояса']).dt.date

In [10]:
data_2['Время создания с час поясом'] = pd.to_datetime(data_2['дата и время создания с учетом час пояса']).dt.time

In [11]:
data_2['День недели'] = pd.to_datetime(data_2['Дата создания с час поясом']).dt.dayofweek

Произведем действия по добавлению дней из столбцов даты и времени прибытия груза. Прибытие груза на выходных - прибавляем количество дней до ближайшего рабочего дня. В пятницу после 12:00 местного времени - аналогично.

In [12]:
data_2['Время создания с час поясом'] = pd.to_datetime(data_2['дата и время создания с учетом час пояса']).dt.hour
def time_lp_1(data_2):
    if ((data_2['День недели'] == 4) and (data_2['Время создания с час поясом'] >= 12)):
        return data_2['дата и время создания с учетом час пояса'] + relativedelta(days=3)
    elif data_2['День недели'] == 5:
        return data_2['time'] + relativedelta(days=2)
    elif data_2['День недели'] == 6:
        return data_2['time'] + relativedelta(days=1)
    else:
        return data_2['Дата создания с час поясом']
data_2['Дата создания с учетом выходных'] = data_2.apply(time_lp_1, axis=1)
data_2['Дата создания с учетом выходных'] = data_2['Дата создания с учетом выходных'].dt.date
data_2['Дата создания с учетом выходных']

0        2023-05-29
1        2023-05-29
2        2023-05-24
3        2023-05-23
4        2023-05-22
            ...    
12616    2023-05-25
12617    2023-05-25
12618    2023-05-26
12619    2023-05-29
12620    2023-05-25
Name: Дата создания с учетом выходных, Length: 12621, dtype: object

In [13]:
data_2['Плановая дата забора'] = data_2['Плановая дата забора'].astype(str)

Нет даты прибытия - не берем в расчет

In [14]:
def time_lp_2(data_2):
    if data_2['Плановая дата забора'] == 'NaT':
        return 'убрать'
    else:
        return ''
data_2['убрать нет план даты'] = data_2.apply(time_lp_2, axis=1)

In [15]:
data_2['Дата статуса  "Груз принят"'] = data_2['Дата статуса  "Груз принят"'].astype(str)

Нет принятия груза на складе от клиента - не берем в расчет

In [16]:
data_2['Плановая дата забора'] = pd.to_datetime(data_2['Плановая дата забора'])
def time_lp_3(data_2):
    if (data_2['Дата статуса  "Груз принят"'] == 'NaT'):
        if (data_2['Плановая дата забора'] > datetime(2023, 5, 30)):
            return 'убрать'
        else:
            return ''
data_2['убрать план дата больше даты создания отчета'] = data_2.apply(time_lp_3, axis=1)

In [17]:
data_2['Дата статуса  "Груз принят"'] = pd.to_datetime(data_2['Дата статуса  "Груз принят"'])

Подсоединим еще одну вкладку к основному датасету, произведем перевод типов столбцов в нужный формат

In [18]:
data_div = data_div.rename(columns={'РП' : 'Наименование завода отправителя'})
data_3 = pd.merge(data_2, data_div, on ='Наименование завода отправителя', how ='left')

In [19]:
data_3['Срок забора без выходных'] = (data_3['Дата статуса  "Груз принят"'] - data_3['Дата создания'])
data_3['Срок забора без выходных'] = data_3['Срок забора без выходных'].dt.days

In [20]:
data_3['Срок забора с выходными'] = (data_3['Дата статуса  "Груз принят"'] - pd.to_datetime(data_3['Дата создания с учетом выходных']))
data_3['Срок забора с выходными'] = data_3['Срок забора с выходными'].dt.days

Получившиеся ошибки при манипуляции с переносом даты прибытия груза, необходимо исправить. Все отрицательные числа заменяем на 0 или положительные

In [21]:
def categorize_ceiling(data_3):
    if data_3['Срок забора с выходными'] == -3:
        return 0
    elif data_3['Срок забора с выходными'] == -2:
        return 1
    elif data_3['Срок забора с выходными'] == -1:
        return 2
    else:
        return data_3['Срок забора с выходными']
data_3['Срок забора с выходными'] = data_3.apply(categorize_ceiling, axis=1)

In [22]:
data_3['меньшее'] = np.minimum(data_3['Срок забора без выходных'], data_3['Срок забора с выходными'])

In [23]:
data_3['меньшее'] = data_3['меньшее'].fillna(-45000)

In [24]:
data_3['убрать нет план даты'] = data_3['убрать нет план даты'].astype(str)
data_3['убрать план дата больше даты создания отчета'] = data_3['убрать план дата больше даты создания отчета'].astype(str)

В зависимости от получившихся чисел в столбце "меньшее" необходимо категоризовать забор груза.

In [25]:
def minimum(data_3):
    if data_3['меньшее'] == 0:
        return 'Забор день в день'
    elif data_3['меньшее'] == 1:
        return 'Забор на следующий день'
    elif data_3['меньшее'] == 2:
        return 'Забор через 2 дня'
    elif data_3['меньшее'] == 3:
        return 'Забор через 3 дня'
    elif data_3['меньшее'] == 4:
        return 'Забор через 4-5 дней'
    elif data_3['меньшее'] == 5:
        return 'Забор через 4-5 дней'
    elif data_3['меньшее'] >= 6:
        return 'Забор через 6 дней и более'
    else:
        return 'Забор груза не состоялся (возможно отсутствует услуга забора)'
data_3['периоды'] = data_3.apply(minimum, axis=1)

Объединим основной датасет со вкладкой списка дополнительных складов и удалим из датасета строки с дополнительными складами.

In [26]:
data_end = pd.merge(data_3, data_dop_scl, on = 'Наименование завода отправителя', how='left')

In [27]:
data_end = data_end[data_end['Наименование завода отправителя'] != ('Минск-2',
'Иваново-2',
'Иваново-3',
'Казань-2',
'Казань-3',
'Нижний Новгород-2',
'Нижний Новгород-3',
'Ижевск-2',
'Пермь-2',
'Екатеринбург-7',
'Екатеринбург-Запад',
'Екатеринбург-КОР',
'Екатеринбург-Север',
'Бишкек-2',
'Бишкек-3',
'Севастополь-2',
'Симферополь-2',
'Владимир-2',
'Пермь-3',
'Саранск-2',
'Тольятти-2',
'Самара-2',
'Ульяновск-2',
'Санкт-Петербург-Восток',
'Санкт-Петербург-Север',
'Санкт-Петербург-Юг',
'Барнаул-2',
'Красноярск-3',
'Новосибирск-2',
'Новосибирск-3',
'Москва-Балашиха',
'Москва-Восток',
'Москва-Запад',
'Москва-Красногорск',
'Москва-Север',                                                             
'Москва-Химки',
'Москва-Щербинка',
'Омск-2',
'Тюмень-2',
'Воронеж-2',
'Анапа-2',
'Краснодар',
'Краснодар-2',
'Ростов-на-Дону-2',
'Челябинск-2',
'Челябинск-3',
'Москва-Дисконт')]

Из нового объединенного датасета уберем строчки, где нет плановой даты и даты, указанной после создания настоящего отчета.

In [28]:
data_end = data_end[data_end['убрать нет план даты'] != 'убрать']
data_end = data_end[data_end['убрать план дата больше даты создания отчета'] != 'убрать']

Составим сводную таблицу отдельно по дивизиональным округам и выведем там столбцы с датой забора и количеством груза.

In [29]:
table = pd.pivot_table(data_end, values='№ ЭР', index='Дивизион',
aggfunc='count', columns='периоды', margins_name='Итог количество', margins=True, dropna=True)

Создадим столбцы с % забора груза по дням.

In [30]:
table['% от количества_1'] = (table['Забор груза не состоялся (возможно отсутствует услуга забора)'] / table['Итог количество']*100).round(2)
table['% от количества_2'] = (table['Забор день в день'] / table['Итог количество']*100).round(2)
table['% от количества_3'] = (table['Забор на следующий день'] / table['Итог количество']*100).round(2)
table['% от количества_4'] = (table['Забор через 2 дня'] / table['Итог количество']*100).round(2)
table['% от количества_5'] = (table['Забор через 3 дня'] / table['Итог количество']*100).round(2)
table['% от количества_6'] = (table['Забор через 4-5 дней'] / table['Итог количество']*100).round(2)
table['% от количества_7'] = (table['Забор через 6 дней и более'] / table['Итог количество']*100).round(2)
table['% от количества_sum'] = (table['Итог количество'] / table['Итог количество']*100).round(2)

Создадим сводную таблицу отдельно по наименованию складов и аналогично предыдущей сводной, проведем действия с данными.

In [31]:
table_2 = pd.pivot_table(data_end, values='№ ЭР', index=['Наименование завода отправителя'],
aggfunc='count', columns='периоды', margins_name='Итог количество', margins=True, dropna=True)

In [32]:
table_2['% от количества_1'] = (table_2['Забор груза не состоялся (возможно отсутствует услуга забора)'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_2'] = (table_2['Забор день в день'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_3'] = (table_2['Забор на следующий день'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_4'] = (table_2['Забор через 2 дня'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_5'] = (table_2['Забор через 3 дня'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_6'] = (table_2['Забор через 4-5 дней'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_7'] = (table_2['Забор через 6 дней и более'] / table_2['Итог количество']*100).round(2)
table_2['% от количества_sum'] = (table_2['Итог количество'] / table_2['Итог количество']*100).round(2)

Распределим столбцы по датасету в соответствии с требованиями компании.

In [33]:
table_2 = table_2[['Забор груза не состоялся (возможно отсутствует услуга забора)', '% от количества_1',
               'Забор день в день', '% от количества_2', 'Забор на следующий день', '% от количества_3',
              'Забор через 2 дня', '% от количества_4', 'Забор через 3 дня', '% от количества_5', 'Забор через 4-5 дней',
               '% от количества_6', 'Забор через 6 дней и более', '% от количества_7', 'Итог количество', '% от количества_sum']]

Объединим обе таблицы в одну. Отчет готов.

In [34]:
table_itog = table_2.append(table)
table_itog = table_itog[['Забор груза не состоялся (возможно отсутствует услуга забора)', '% от количества_1',
               'Забор день в день', '% от количества_2', 'Забор на следующий день', '% от количества_3',
              'Забор через 2 дня', '% от количества_4', 'Забор через 3 дня', '% от количества_5', 'Забор через 4-5 дней',
               '% от количества_6', 'Забор через 6 дней и более', '% от количества_7', 'Итог количество', '% от количества_sum']]

C:\Users\telez\AppData\Local\Temp\ipykernel_12080\1407636483.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_itog = table_2.append(table)
